In [1]:
import sys
from pathlib import Path
p = Path('.').absolute().parent
if str(p) not in sys.path:
    sys.path.append(str(p))

from graph_irl.buffer_v2 import GraphBuffer
from graph_irl.policy import GaussPolicy, TwoStageGaussPolicy, GCN, Qfunc
from graph_irl.graph_rl_utils import GraphEnv
from graph_irl.sac import SACAgentGraph, TEST_OUTPUTS_PATH
from graph_irl.reward import GraphReward
from graph_irl.examples.circle_graph import create_circle_graph
from graph_irl.irl_trainer import IRLGraphTrainer

import random
import numpy as np
import torch
torch.set_printoptions(precision=8)
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)


def get_params(compute_rewards_online):
    # circular graph with 7 nodes;
    n_nodes, node_dim = 11, 5
    nodes, expert_edge_index = create_circle_graph(n_nodes, node_dim, torch.randn)
    # nodes = torch.ones_like(nodes)  # doesn't seem to work for ones;
    # print(nodes, expert_edge_index)
    encoder_hiddens = [8, 8, 8]
    reward_fn_hiddens = [16, 16]
    gauss_policy_hiddens = [16, 16]
    tsg_policy_hiddens1 = [16, 16]
    tsg_policy_hiddens2 = [16]
    qfunc_hiddens = [16, 16]

    # print(f"IRL training for {n_nodes}-node graph")

    encoder_dict = dict(
        encoder = GCN(node_dim, encoder_hiddens, with_layer_norm=True, final_tanh=True),
        encoderq1 = GCN(node_dim, encoder_hiddens, with_layer_norm=True, final_tanh=True),
        encoderq2 = GCN(node_dim, encoder_hiddens, with_layer_norm=True, final_tanh=True),
        encoderq1t = GCN(node_dim, encoder_hiddens, with_layer_norm=True, final_tanh=True),
        encoderq2t = GCN(node_dim, encoder_hiddens, with_layer_norm=True, final_tanh=True),
        encoder_reward = GCN(node_dim, encoder_hiddens, with_layer_norm=True, final_tanh=True),
    )

    reward_fn = GraphReward(
        encoder_dict['encoder_reward'], 
        encoder_hiddens[-1], 
        hiddens=reward_fn_hiddens, 
        with_layer_norm=True,
        with_batch_norm=False,
    )

    gauss_policy_kwargs = dict(
        obs_dim=encoder_hiddens[-1],
        action_dim=encoder_hiddens[-1],
        hiddens=gauss_policy_hiddens,
        with_layer_norm=True,
        encoder=encoder_dict['encoder'],
        two_action_vectors=True,
    )

    tsg_policy_kwargs = dict(
        obs_dim=encoder_hiddens[-1],
        action_dim=encoder_hiddens[-1],
        hiddens1=tsg_policy_hiddens1,
        hiddens2=tsg_policy_hiddens2,
        encoder=encoder_dict['encoder'],
        with_layer_norm=True,
    )

    qfunc_kwargs = dict(
        obs_action_dim=encoder_hiddens[-1] * 3, 
        hiddens=qfunc_hiddens, 
        with_layer_norm=True, 
        with_batch_norm=False,
        encoder=None
    )

    Q1_kwargs = qfunc_kwargs.copy()
    Q1_kwargs['encoder'] = encoder_dict['encoderq1']
    Q2_kwargs = qfunc_kwargs.copy()
    Q2_kwargs['encoder'] = encoder_dict['encoderq2']
    Q1t_kwargs = qfunc_kwargs.copy()
    Q1t_kwargs['encoder'] = encoder_dict['encoderq1t']
    Q2t_kwargs = qfunc_kwargs.copy()
    Q2t_kwargs['encoder'] = encoder_dict['encoderq2t']

    agent_kwargs=dict(
        name='SACAgentGraph',
        policy_constructor=GaussPolicy,
        qfunc_constructor=Qfunc,
        env_constructor=GraphEnv,
        buffer_constructor=GraphBuffer,
        optimiser_constructors=dict(
            policy_optim=torch.optim.Adam,
            temperature_optim=torch.optim.Adam,
            Q1_optim=torch.optim.Adam,
            Q2_optim=torch.optim.Adam,
        ),
        entropy_lb=encoder_hiddens[-1],
        policy_lr=3e-4,
        temperature_lr=3e-4,
        qfunc_lr=3e-4,
        tau=0.01,
        discount=1.,
        save_to=TEST_OUTPUTS_PATH,
        cache_best_policy=False,
        clip_grads=False,
        zero_temperature=False,
        UT_trick=False,
        with_entropy=False,
    )

    config = dict(
        training_kwargs=dict(
            seed=0,
            num_iters=31,
            num_steps_to_sample=100,
            num_grad_steps=1,
            batch_size=11,
            num_eval_steps_to_sample=n_nodes,
            min_steps_to_presample=0,
        ),
        Q1_kwargs=Q1_kwargs,
        Q2_kwargs=Q2_kwargs,
        Q1t_kwargs=Q1t_kwargs,
        Q2t_kwargs=Q2t_kwargs,
        policy_kwargs=gauss_policy_kwargs,
        buffer_kwargs=dict(
            max_size=10_000,
            nodes=nodes,
            seed=0,
            drop_repeats_or_self_loops=True,
            get_batch_reward=True,
            compute_rewards_online=compute_rewards_online,
            graphs_per_batch=100,
            action_is_index=True,
            per_decision_imp_sample=True,
            reward_scale=encoder_hiddens[-1] * 2,
            log_offset=0.,
        ),
        env_kwargs=dict(
            x=nodes,
            reward_fn=reward_fn,
            max_episode_steps=n_nodes,
            num_expert_steps=n_nodes,
            max_repeats=n_nodes,
            max_self_loops=n_nodes,
            drop_repeats_or_self_loops=True,
            id=None,
            reward_fn_termination=False,
            calculate_reward=False,
            min_steps_to_do=3,
        )
    )
    return agent_kwargs, config, nodes, expert_edge_index


agent_kwargs, config, nodes, expert_edge_index = get_params(
    compute_rewards_online=True
)

agent = SACAgentGraph(
    **agent_kwargs,
    **config
)

irl_trainer_config = dict(
    num_expert_traj=10,
    graphs_per_batch=config['buffer_kwargs']['graphs_per_batch'],
    num_extra_paths_gen=5,
    reward_optim_lr_scheduler=None,
    reward_grad_clip=False,
    reward_scale=config['buffer_kwargs']['reward_scale'],
    per_decision_imp_sample=config['buffer_kwargs']['per_decision_imp_sample'],
    unnorm_policy=True,
    add_expert_to_generated=False,
    lcr_regularisation_coef=None,
    mono_regularisation_on_demo_coef=1 / (expert_edge_index.shape[-1] // 2),
    verbose=False,
)

irl_trainer = IRLGraphTrainer(
    reward_fn=agent.env.reward_fn,
    reward_optim=torch.optim.Adam(agent.env.reward_fn.parameters(), lr=1e-2),
    agent=agent,
    nodes=nodes,
    expert_edge_index=expert_edge_index,
    **irl_trainer_config,
)

irl_trainer.reward_fn.requires_grad_(True)
irl_trainer.reward_fn.train()
irl_trainer.agent.policy.requires_grad_(False)
irl_trainer.do_reward_grad_step()

pygame 2.1.3 (SDL 2.0.22, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html
/home/focal/coding/urban-nets-style-transfer/tests
expert return: -122.74497985839844
expert return: -149.66355895996094
expert return: -124.43367004394531
expert return: -146.4972686767578
expert return: -138.7230682373047
expert return: -153.1378173828125
expert return: -139.2919158935547
expert return: -134.63589477539062
expert return: -135.19525146484375
expert return: -153.7252960205078
expert_rewards shape:  torch.Size([10, 11])
sampled undiscounted return: -137.8103790283203
sampled undiscounted return: -143.25941467285156
sampled undiscounted return: -142.55328369140625
sampled undiscounted return: -150.7525177001953
sampled undiscounted return: -119.78134155273438
sampled undiscounted return: -147.7942657470703
sampled undiscounted return: -134.21124267578125
sampled undiscounted return: -147.94541931152344
sampled undiscounted return: -154.61767578125
sampled undi

In [2]:
irl_trainer.reward_fn.requires_grad_(False)
irl_trainer.reward_fn.eval()
irl_trainer.agent.policy.requires_grad_(True)
irl_trainer.agent.policy.eval()
irl_trainer.train_policy_k_epochs(1)

  0%|          | 0/1 [00:00<?, ?it/s]

bob is: 110
tensor([[10, 10,  4,  0,  9,  2,  1,  3,  6,  8,  4],
        [ 1,  7,  9,  8,  8,  0,  4,  0,  7,  9,  3]])
tensor([ -5.01699448, -10.65846252, -10.90252399,  -9.80887985,  -9.80794048,
        -20.05060196,  -6.02020884, -12.78817081,  -9.87467861, -18.93912888,
         -8.24656963])



bob is: 110
tensor([[0, 3, 7, 2, 4, 2, 0, 1, 2, 2, 9],
        [6, 0, 0, 8, 5, 0, 3, 2, 1, 8, 0]])
tensor([-13.58244228,  -8.29182720, -16.62692833, -20.31767845, -14.35860252,
        -21.08855629, -11.96698666,  -8.06600475,  -7.78110695, -20.34567451,
        -12.27365112])



bob is: 110
tensor([[ 1,  5,  2, 10,  0,  3,  0,  9,  9,  1,  5],
        [10, 10, 10,  6,  7, 10, 10,  1,  4,  7,  2]])
tensor([-13.74940014, -17.23501205, -19.97656631, -10.68769932, -13.47595119,
        -10.56976509,  -7.67417145,  -7.25369453,  -9.85009289, -21.47320938,
         -6.64267731])



bob is: 110
tensor([[ 5,  1,  6,  1,  1,  9,  5,  8,  9,  1,  2],
        [ 3,  4,  7,  0,  0,  7,  7, 10,  5,  7, 10]])
tensor([-10.35365200,  -6.02020884, -13.11557293, -14.72109318, -13.87147427,
        -10.26946354, -10.41771317, -14.30883026,  -8.26774216, -15.71107006,
        -19.73834038])



...

bob is: 110
tensor([[ 1,  2, 10,  0,  3,  2,  2,  6,  3,  9,  7],
        [ 8,  6,  4,  5,  6, 10,  0,  0,  8,  1,  2]])
tensor([-15.14312744, -19.86323547,  -9.75464344, -11.57705688, -14.14843082,
        -19.07562637, -20.66304398, -12.06444740, -14.62690163,  -9.18458271,
        -10.55954170])



100%|██████████| 1/1 [00:13<00:00, 13.44s/it]

bob is: 110
tensor([[ 5,  9, 10,  6,  7,  6,  1,  7,  2,  6, 10],
        [ 7, 10,  8,  7,  9,  0,  7, 10,  7,  8,  8]])
tensor([-20.30950546,  -8.11925697, -13.98352432, -10.59698582, -14.32473755,
        -11.60579395, -15.71107006, -10.25322914, -16.59001732, -11.71584034,
        -15.57777596])

3409



/home/focal/coding/urban-nets-style-transfer/graph_irl/vis_utils.py:90: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax.set_ylim(ylow, yhigh)


3409 [[10  8]
 [ 0  8]
 [10  6]
 [ 2  6]
 [ 5  0]] [[ 0  9]
 [ 0  6]
 [ 4 10]
 [ 5  0]
 [ 8  7]] [Data(x=[11, 5], edge_index=[2, 0]), Data(x=[11, 5], edge_index=[2, 2]), Data(x=[11, 5], edge_index=[2, 4]), Data(x=[11, 5], edge_index=[2, 6]), Data(x=[11, 5], edge_index=[2, 8])] [Data(x=[11, 5], edge_index=[2, 12]), Data(x=[11, 5], edge_index=[2, 14]), Data(x=[11, 5], edge_index=[2, 16]), Data(x=[11, 5], edge_index=[2, 18]), Data(x=[11, 5], edge_index=[2, 20])]


In [3]:
irl_trainer.reward_fn.requires_grad_(True)
irl_trainer.reward_fn.train()
irl_trainer.agent.policy.requires_grad_(False)
irl_trainer.do_reward_grad_step()

expert return: -114.7294921875
expert return: -116.73078155517578
expert return: -108.88362121582031
expert return: -126.7417221069336
expert return: -112.21012878417969
expert return: -117.59815979003906
expert return: -123.91873931884766
expert return: -108.78541564941406
expert return: -120.0748519897461
expert return: -130.11383056640625
expert_rewards shape:  torch.Size([10, 11])
sampled undiscounted return: -139.0196075439453
sampled undiscounted return: -149.67205810546875
sampled undiscounted return: -145.4356231689453
sampled undiscounted return: -141.41696166992188
sampled undiscounted return: -122.41036224365234
sampled undiscounted return: -135.726318359375
sampled undiscounted return: -137.29229736328125
sampled undiscounted return: -146.5854034423828
sampled undiscounted return: -133.09445190429688
sampled undiscounted return: -125.06513214111328
sampled undiscounted return: -147.83804321289062
sampled undiscounted return: -127.80394744873047
sampled undiscounted return: 

In [4]:
torch.nn.utils.parameters_to_vector(irl_trainer.reward_fn.parameters())

tensor([-1.13003738e-02,  1.24404430e-02, -1.24041177e-02, -1.72416717e-02,
        -1.87604986e-02,  1.32125989e-02,  1.66871380e-02,  1.68949608e-02,
         5.25503457e-01,  5.49489796e-01,  3.66495341e-01,  2.65446931e-01,
        -1.83788076e-01,  3.75868529e-01,  3.17790210e-01,  2.16946438e-01,
        -1.22462017e-02,  1.07374601e-01, -1.60819143e-02, -4.78162974e-01,
         1.71968445e-01, -5.08480251e-01,  3.20569426e-01, -4.21342403e-01,
        -2.06193551e-01, -3.05994183e-01,  5.46577871e-01, -4.95204031e-01,
         3.64286751e-01,  3.60076994e-01,  4.97049958e-01,  1.51331976e-01,
         3.51813316e-01,  5.52480936e-01,  1.22800104e-01,  2.19568774e-01,
        -5.11437356e-01, -1.65760592e-01,  5.66379614e-02,  6.58650845e-02,
        -2.56528825e-01, -6.80525661e-01, -1.64700508e-01,  2.76522189e-01,
         5.68592191e-01, -6.29678607e-01,  3.43454599e-01, -6.29876316e-01,
         1.71520747e-02,  1.89763866e-02, -1.84109267e-02,  1.78483278e-02,
         1.6

In [3]:
import numpy as np
import torch
torch.set_float32_matmul_precision("highest")

rng = np.random.RandomState(1)
dataset1 = rng.uniform(low=-0.01, high=0.01, size=(1000, 20)).astype(np.float32)
dataset2 = torch.from_numpy(dataset1)
print(dataset1.mean(), dataset1.std())
print(dataset2.mean(), dataset2.std())

2.5537092e-05 0.005769608
tensor(2.5537e-05) tensor(0.0058)
